In [5]:
import pandas as pd
import numpy as np
df= pd.read_csv('HIV smalldata.csv')
df

,SMILES,ACTIVITY
0,C1[C@H]2[C@@H]([C@@H](S1)CCCCC(=O)O)NC(=O)N2,1
1,C1=COC(=C1)/C=C/C=C\2/C(=O)N(C(=S)S2)C(CC(=O)O...,1
2,C1=CC2=C(C(=C1)C3=C4C=CC(=O)C(=C4OC5=C3C=CC(=C...,1
3,C1=CC(=C(C=C1C[C@@H](C(=O)O)N)O)O,1
4,C1C2=CC(=C(C=C2C3=C4C=CC(=O)C(=C4OCC31O)O)O)O,1
...,...,...
2015,C1COCCN1C2=CC=C(C=C2)NCC3=CC4=C(C=C3)OCO4.C(=O...,0
2016,C1=C(C=C(C(=C1/C=N/NC2=NN=CN2N)O)Br)Br.Cl,0
2017,C[C@H]1CN(C(=O)CCCN2C(=CN=N2)CO[C@@H]1CN(C)S(=...,0
2018,CCCCCOC(=O)CC(C(=O)OCCCCC)N1CCCCC1.Cl,0


In [6]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors
from rdkit import DataStructs
import numpy as np
import pandas as pd

In [7]:
smiles_list = df['SMILES'].tolist() 
mols = [Chem.MolFromSmiles(smi) for smi in smiles_list if smi is not None]

In [8]:
def morgan_fps(data):
  fps = [AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048) for mol in data]  # Calculate Morgan fingerprints for each molecule
  fp_array = [np.array(fp) for fp in fps]   # Convert fingerprints to numpy array
  column_names = ["morgan_" + str(i) for i in range(len(fp_array[0]))]
  return pd.DataFrame(fp_array, columns = column_names)

In [9]:
d1 = morgan_fps(mols)
X = pd.concat([d1], axis = 1)
X

,morgan_0,morgan_1,morgan_2,morgan_3,morgan_4,morgan_5,morgan_6,morgan_7,morgan_8,morgan_9,...,morgan_2038,morgan_2039,morgan_2040,morgan_2041,morgan_2042,morgan_2043,morgan_2044,morgan_2045,morgan_2046,morgan_2047
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2017,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018,0,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [10]:
y = df["ACTIVITY"]
y

0       1
1       1
2       1
3       1
4       1
       ..
2015    0
2016    0
2017    0
2018    0
2019    0
Name: ACTIVITY, Length: 2020, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 8)

In [12]:
from imblearn.over_sampling import SMOTE

# assuming X_train and y_train are your training features and labels

smote = SMOTE(sampling_strategy='minority',random_state=2)

X_sm, y_sm = smote.fit_resample(X_train, y_train)
X_train = X_sm
y_train = y_sm

In [13]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)

# Fit and transform the data
sel.fit(X_train)

mask = sel.get_support()
X = X_train[X_train.columns[mask]]

In [29]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=4)
gbm.fit(X_sm, y_sm)

preds = gbm.predict(X_test)
proba = gbm.predict_proba(X_test)

In [30]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
accuracy = (tn + tp) / (tn+tp+fn+fp)
balanced_accuracy = (sensitivity + specificity) / 2
precision = tp / (tp + fp)
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
roc_auc = roc_auc_score(y_test, proba[:, 1])

print(f"sensitivity: {sensitivity}, specificity: {specificity}, precision: {precision}, f1: {f1} , accuracy: {accuracy}, roc_auc: {roc_auc} \nbalanced_accuracy: {balanced_accuracy}")


sensitivity: 0.636986301369863, specificity: 0.8333333333333334, precision: 0.6838235294117647, f1: 0.6595744680851064 , accuracy: 0.7623762376237624, roc_auc: 0.8268556865243709 
balanced_accuracy: 0.7351598173515982


In [35]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9]
}

# Initialize the classifier
gbm = GradientBoostingClassifier(random_state=4)

# Initialize Grid Search
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit GridSearch to the data
grid_search.fit(X_test, y_test)

# Print the best parameters
print('Best parameters found: ', grid_search.best_params_)


Best parameters found:  {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200}


In [36]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_features = 5 , n_estimators = 100, random_state=2)
rfc.fit(X_train, y_train)
#rfc.fit(X_train, y_train)

# Make predictions on the testing set
preds = rfc.predict(X_test)
proba = rfc.predict_proba(X_test)

In [37]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
accuracy = (tn + tp) / (tn+tp+fn+fp)
balanced_accuracy = (sensitivity + specificity) / 2
precision = tp / (tp + fp)
f1 = 2 * (precision * sensitivity) / (precision + sensitivity)
roc_auc = roc_auc_score(y_test, proba[:, 1])

print(f"sensitivity: {sensitivity}, specificity: {specificity}, precision: {precision}, f1: {f1} , accuracy: {accuracy}, roc_auc: {roc_auc} \nbalanced_accuracy: {balanced_accuracy}")

sensitivity: 0.7191780821917808, specificity: 0.8255813953488372, precision: 0.7, f1: 0.7094594594594595 , accuracy: 0.7871287128712872, roc_auc: 0.8549431878517575 
balanced_accuracy: 0.7723797387703091
